In [163]:
# Pyspark initialization and session creation

import findspark
findspark.init()
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

import gensim
# from pyspark.ml.feature import HashingTF, IDF, Tokenizer, Normalizer, CountVectorizer
from pyspark.mllib.feature import HashingTF, IDF
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.ml.linalg import DenseVector, VectorUDT

from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS

conf = SparkConf()

spark = SparkSession \
    .builder \
    .appName('Word2Vec Implementation') \
    .getOrCreate()

spark.conf.set('spark.sql.caseSensitive', False)


In [164]:
#####clean data
documents = ['over the last two decades tremendous advances have been made for constructing large scale quantum computers in particular quantum computing platforms based on superconducting qubits have become the leading candidate for scalable quantum processor architecture and the milestone of demonstrating quantum supremacy has been first achieved using superconducting qubits in in this study we provide a brief review on the experimental efforts towards the large scale superconducting quantum computer including qubit design quantum control readout techniques and the implementations of error correction and quantum algorithms besides the state of the art we finally discuss future perspectives and which we hope will motivate further research ',
             'the multidisciplinary approach to breast cancer treatment in a comprehensive breast center model can deliver a high standard of care for the patient but it also generates a wide range of patient data that are typically left as disparate silos and not organized or analyzed for any clinician to help with patient care we have developed a method for the synthesis of radiology surgery pathology medical oncology and genetics information from a patient medical history in a timeline paradigm with elements of clinical decision support and quality indicators to help with decision making a model for structuring data from the many dispar ate sources was developed to support enriched probing interactions summary views of the current knowledge from each discipline were also tightly integrated to support the surgical management workflow the prototype lifelines were integrated into existing clinical software incorporating all the relevant events decisions and data elements keywords patient timeline electronic health records clinical decision support', 
             'today computing power is advancing with significant transformation in quantum technologies and provides state of the art functionalities for next generation digital solutions at the same time computing capabilities are influencing the various spheres of humanity while touching the operating models and end user experiences the advent of quantum computing raised many questions about the system design development implementation and associated risks for human beings and species the questions raised the call for a conceptual model to study ethics and quantum computing as a whole the primary objective of the research is to develop a conceptual ethical and sustainable quantum computing model to investigate the ethics standards trust levels social aspects access rights privacy moral and good duties issues based on the current situations the paper adopts the literature survey approach based on existing business ethics models to evaluate the model s human organisation society and opportunity factors the research has investigated the ethical constitutes of quantum computing led digital transformation the research findings provide the substantial foundations for the research and managerial applicability to integrate the social human technology based inclusive ethical issues keywords quantum computing business ethics ethical model digital transformation sustainability ', 
             'see discussions st ats and author pr ofiles f or this public ation at https www researchgate ne t public ation review on cryptography using quantu m computing\narticle in international journal f or modern t rends in scienc e and t echnolog y januar y doi ijmt st\ncitations\nreads author s including edit or ijmtst\ninternational journal f or modern t rends in scienc e and t echnolog y publica tions citations see profile\nall c ontent f ollo wing this p age was uplo aded b y edit or ijmtst on januar y the user has r equest ed enhanc ement of the do wnlo aded file ', 
             'cancer is a fatal disease and one of the leadingcauses of death worldwide the cure rate in cancer treatmentremains low hence cancer treatment is gradually shifting towardpersonalized treatment arti cial intelligence ai and radiomicshave been recognized as one of the potential areas of researchin personalized medicine in oncology several researchers haveidenti ed the capabilities of ai and radiomics to character ize phenotype and there by predict the outcome of treatmentin oncology although ai and radiomics have shown promis ing initial results in diagnosis and treatment in oncology thesetechnologies are also facing challenges of standardization andscalability in the last few years researchers have been trying todevelop a research infrastructure for federated machine learn ing that increases the usability of big data for clinical research these research infrastructures are based on the ndable acces sible interoperable and reusable i e fair data principles the india dutch big imaging data approach for oncology ina netherlands india collaboration bionic is a jointly fundedinitiative by the dutch research council nwo and the indianministry of electronics and information technology meity aiming to introduce radiomic based research into clinical envi ronments using federated machine learning on geographicallydispersed collections of fair data this article described a proto type end to end research infrastructure implemented through thebionic partnership into a leading cancer care public hospitalin india index terms arti cial intelligence ai ndable accessi ble interoperable and reusable fair data machine learning natural language processing nlp radiomics manuscript received may revised september acceptedseptember date of publication september date ofcurrent version february this work was supported in part bynwo in netherlands sanction no and in part by theministry of electronics and information technology in india sanction no cc bt leonard wee v rangarajan johan van soest andandre dekker are contributed equally to this work corresponding author ashish kumar jha this work involved human subjects or animals in its research approvalof all ethical and experimental procedures and protocols was granted bythe hospital institutional ethics committee institutional ethics committee i tata memorial centre iec tmc mumbai india under applicationno iec as a retrospective study with waivers of informedconsent from involved patients as per iec policy of our hospital by the sameethics committee please see the acknowledgment section of this article for the authoraf liations this article has supplementary material provided by theauthors and color versions of one or more gures available athttps doi org trpms digital object identi er trpms i i', 
             'bladder cancer bc is the most common urinary malignancy however accurate diagnosis\nand prediction of recurrence after therapies remain elusive this study aimed to develop a biosignature of\nimmunotherapy based responses using gene expression data publicly available bc datasets were collected and machine learning ml approaches were applied to identify a novel biosignature to differentiate patient\nsubgroups immune phenotyping of bc in the imvigor dataset included three subtypes in amed excluded and desert immune immune phenotypes were analyzed with gene expressions using traditional\nbut powerful classi cation methods such as random forests deep neural networks dnn support vector\nmachines svm together with boosting and feature selection methods speci cally dnn yielded the\nhighest area under the curve auc with precision and recall pr curves and receiver operating charac teristic roc curves for each phenotype and respectively resulting in the\nidenti cation of gene expression features useful for immune phenotype classi cation our results suggest\nsigni cant potential to further develop and utilize machine learning algorithms for analysis of bc and\nits precaution in conclusion the ndings from this study present a novel gene expression assay that can\naccurately discriminate bc patients from controls upon further validation in independent cohorts this gene\nsignature could be developed into a predictive test that can support clinical evaluation and patient care index terms arti cial algorithm biomarker bladder cancer gene expression immunotherapy machine\nlearning impact statement machine learning approaches to predict the immune phenotypes in bladder\ncancer patients to develop a biosignature of immunotherapy based responses using gene expression\ndata deep neural networks dnn support vector machines svm together with boosting and feature\nselection methods were applied dnn yielded the highest area under the curve auc with receiver operating\ncharacteristic roc curves and precision and recall pr curves for each phenotype and respectively our results suggest signi cant potential to further develop and utilize machine learning\nalgorithms for analysis of bladder cancer and its precaution this work is licensed under a creative commons attribution license for more information see http creativecommons org licenses by volume ', 
             'the wide adoption of electronic health records ehrs provides a potential source to support clinical research lung cancer is one of the most common cancers in the world although several tools have been developed to automaticallyextract medical concepts from clinical notes there is still a gapbetween concept extraction and concept understanding the highnumber of clinical notes written for one single patient the useof negation speculation and proper date annotations lay at theroot of the problem in this paper we propose an approachto obtain more accurate lung cancer diagnosis extraction fromclinical notes written in spanish the approach deals with adisambiguation process that is required to extract the correctdate and diagnosis of a patient having hundreds of clinical notesand consequently hundreds of annotations results obtained onan annotated database of patients show an f score of index t erms natural language processing nlp informa tion extraction lung cancer diagnosis diagnosis extraction i i', 
             'quantum computing qc is considered as a rising star of computing technologi es with very promising possibilities towards novel solutions of even more complex computing tasks than tho se which are tackled using today s supercomputers however direct use in everyday applications is lac king both large scale quantum computers and quantum algorithms i e software we are carrying out the first proj ect on a road towards qc enabled computed tomography ct in our paper we present this project after its first out of five years duration and share the first steps we made with the community it is worth to mention that the hardware for qc is still in a phase of deve lopment which implies that most of the software research is in a phase of becoming ready f or productional use cases ',
             'starting from a recursive construction of the natural numbers a cartesian sum of prime numbers with the elements of the set of powers of is described boolean and logical matrices generated through recurring cartesian sums permit the defini tion of extended twin primes that is primes separated by a gap of a power n for a natural number n are studied besides the set of extended twin primes through computational techniques that were inspired by relativistic quantum two component spinor computations we demonstrate the existence of two other characteristic types of primes which we call iso boolean and singular primes respectively the iso boolean primes possess rows in the boolean matrix composed of the same elements of the canonical basis of a boolean hypercube the singular primes exhibit a zero boolean matrix row and it is computationally established that there are numerous such singular and iso boolean primes all of these novel findings are formulated as three new conjectures on extended twin primes keywords recursive construction of natural number set cartesian sum of primes with powers of boolean matrices of a hypercube isoboolean and singular primes extended twin primes conjecture relativistic quantum spinor computations krishnan balasubramanian baluk edu ramon carb dorca ramoncarbodorca com school of molecular sciences arizona state university tempe az usa institut de qu mica computacional i cat lisi universitat de girona girona catalonia spain and ronin institute montclair nj usa', 
             'this paper outlines the major components and func tion of the technologically integrated oncosimulator developed pri marily within the advancing clinico genomic trials on cancer acgt project the oncosimulator is de ned as an information\ntechnology system simulating in vivo tumor response to therapeutic\nmanuscript received february revised july accepted\nseptember date of publication october date of current version\nmay this work was supported in part by the european commission\nunder the project acgt advancing clinicogenomic trials on cancer fp ist project contra cancrum clinically oriented translational\ncancer multilevel modeling fp ict project tumor transatlantic tumor model repositories fp ict project p medicine personalized medicine fp ict and project chic computational horizons in cancer chic developing meta and\nhyper multiscale models and repositories for in silico oncology fp ict g s stamatakos corresponding author is with institute of communica tion and computer systems national technical university of athens in silico\noncology group iroon polytechniou gr zografos greece e mail gestam ntua gr d dionysiou e kolokotroni e georgiadi and s giatili are with the\nin silico oncology group institute of communication and computer sys tems national technical university of athens gr zografos greece e mail dimdio ece ntua gr ekolok ntua gr egeorg ntua gr giatili gr a lunzer was with hokkaido university sapporo japan he is\nnow with alan kay s viewpoints research institute los angeles ca usa e mail aranlunzer com r belleman is with the university of amsterdam wx amsterdam the netherlands e mail r g belleman nl m erdt is with the fraunhofer institute darmstadt germany e mail marius erdt fraunhofer de j pukacki is with the poznan supercomputing and networking center psnc poznan poland e mail pukacki poznan pl s rueping is with the fraunhofer iais schloss birlinghoven st augustin germany e mail stefan rueping fraunhofer de a d onofrio is with the istituto europeo di oncologia milano italy e mail alberto donofrio it s sfakianakis and k marias are with the foundation for research and\ntechnology hellas heraklion greece e mail ssfak forth gr kmarias forth gr c desmedt is with the institut jules bordet centre des tumeurs brux elles belgium e mail christine desmedt be m tsiknakis is with the department of informatics engineering tei crete\nand the computational medicine laboratory institute of computer science forth heraklion greece e mail tsiknaki teicrete gr n graf is with the university hospital of the saarland pediatric\nhaematology and oncology d homburg germany e mail norbert graf saarland de color versions of one or more of the gures in this paper are available online\nat http ieeexplore ieee org digital object identi er jbhi modalities within the clinical trial context chemotherapy in the\nneoadjuvant setting according to two real clinical trials concern ing nephroblastoma and breast cancer has been considered the\nspatiotemporal simulation module embedded in the oncosimula tor is based on the multiscale predominantly top down discrete\nentity discrete event cancer simulation technique developed by\nthein silico oncology group national technical university of\nathens the technology modules include multiscale data handling image processing invocation of code execution via a spreadsheet inspired environment portal execution of the code on the grid and the visualization of the predictions a re ning scenario for\nthe eventual coupling of the oncosimulator with immunological\nmodels is also presented parameter values have been adapted to\nmultiscale clinical trial data in a consistent way thus supporting the\npredictive potential of the oncosimulator indicative results demon strating various aspects of the clinical adaptation and validation\nprocess are presented completion of these processes is expected to\npave the way for the clinical translation of the system index terms advancing clinico genomic trials on cancer acgt project breast cancer in silico medicine in silico oncol ogy multiscale cancer modeling nephroblastoma oncosimulator virtual physiological human vph ', 
             'brain tumor detection from magnetic resonance mr images is a tedious task but vital for early prediction of the disease which until now is solely based on the experience of medical practitioners multilevel image segmentation is a computationally simple and efficient approach for segmenting brain mr images conventional image segmentation does not consider the spatial correlation of image pixels and lacks better post filtering efficiency this study presents a renyi entropy based multilevel image segmentation approach using a combination of differential evolution and whale optimization algorithms dewo to detect brain tumors further to validate the efficiency of the proposed hybrid algorithm it is compared with some prominent metaheuristic algorithms in recent past using between class variance and the tsallis entropy functions the proposed hybrid algorithm for image segmentation is able to achieve better results than all the other metaheuristic algorithms in every entropy based segmentation performed on brain mr images keywordsd histogram between class variance brain mr image segmentation multilevel thresholding whale optimization', 
             'in this paper we apply the dempster shafer theory to outcome analysis in radiation oncology evidence from different sources some of them inconsistent is combined by dempster s rule to provide an innovative method for radiotherapy outcome analysis keywords evidence dempster shafer theory radiation oncology pneumonitis i ', 
             'see discussions st ats and author pr ofiles f or this public ation at https www researchgate ne t public ation analyzing the data of software secu rity life span quantu m computing era\npreprint in intellig ent a utomation and soft computing sept ember doi iasc citations\nreads author s including some o f the author s of this public ation ar e also w orking on these r elat ed pr ojects fake ne ws de tection view pr oject\nmanaging softw are security risk a design me trics p erspectiv e view pr oject\nmohd nadeem\nbab asaheb bhimr ao ambedk ar univ ersity publica tions citations see profile\nwael alosaimi\ntaif univ ersity publica tions citations see profile\nabdullah alharbi\nuniv ersity of t abuk publica tions citations see profile\nrajee v kumar\nadministr ativ e st aff colle ge of india publica tions citations see profile\nall c ontent f ollo wing this p age was uplo aded b y mohd nadeem on sept ember the user has r equest ed enhanc ement of the do wnlo aded file ', 
             'work in progress the theory of quantum me chanics qm provides a foundation for many fields of science and engineering however its abstract nature and technical diffi culty make qm a challenging subject for students to approach and grasp this is partly because complex mathematical concepts involved in qm are difficult to visualize for students and the ex isting visualization are minimal and limited we propose that many of these concepts can be communicated and experienced through interactive visualizations and games drawing on the strengths and affordances of digital media a game environment can make qm concepts more accessible and understandable by immersing students in nano sized worlds governed by unique qm rules furthermore replayability of games allows students to experience the probabilistic nature of qm concepts in this paper we present a game and a series of interactive visualizations that we are developing to provide students with an experiential envi ronment to learn quantum mechanics we will discuss how these visualizations and games can enable students to experiment with qm concepts compare qm with classical physics and get accus tomed to the often counterintuitive laws of qm keywords quantum mechanics semiconductor physics edu cation scientific visualization games',
             'cancer is a fatal disease and one of the leadingcauses of death worldwide the cure rate in cancer treatmentremains low hence cancer treatment is gradually shifting towardpersonalized treatment arti cial intelligence ai and radiomicshave been recognized as one of the potential areas of researchin personalized medicine in oncology several researchers haveidenti ed the capabilities of ai and radiomics to character ize phenotype and there by predict the outcome of treatmentin oncology although ai and radiomics have shown promis ing initial results in diagnosis and treatment in oncology thesetechnologies are also facing challenges of standardization andscalability in the last few years researchers have been trying todevelop a research infrastructure for federated machine learn ing that increases the usability of big data for clinical research these research infrastructures are based on the ndable acces sible interoperable and reusable i e fair data principles the india dutch big imaging data approach for oncology ina netherlands india collaboration bionic is a jointly fundedinitiative by the dutch research council nwo and the indianministry of electronics and information technology meity aiming to introduce radiomic based research into clinical envi ronments using federated machine learning on geographicallydispersed collections of fair data this article described a proto type end to end research infrastructure implemented through thebionic partnership into a leading cancer care public hospitalin india index terms arti cial intelligence ai ndable accessi ble interoperable and reusable fair data machine learning natural language processing nlp radiomics manuscript received may revised september acceptedseptember date of publication september date ofcurrent version february this work was supported in part bynwo in netherlands sanction no and in part by theministry of electronics and information technology in india sanction no cc bt leonard wee v rangarajan johan van soest andandre dekker are contributed equally to this work corresponding author ashish kumar jha this work involved human subjects or animals in its research approvalof all ethical and experimental procedures and protocols was granted bythe hospital institutional ethics committee institutional ethics committee i tata memorial centre iec tmc mumbai india under applicationno iec as a retrospective study with waivers of informedconsent from involved patients as per iec policy of our hospital by the sameethics committee please see the acknowledgment section of this article for the authoraf liations this article has supplementary material provided by theauthors and color versions of one or more gures available athttps doi org trpms digital object identi er trpms i i', 
             'segmentation is consider as one of the main step in image processing and it plays and importan t role in image processing it is the process of subdividing an image into its constituent parts in this paper we have reviewed various methods of segmentation and its application in medical image processing i e mri i mage ultrasound image etc we have focused on brain tumor mri image recent medical imaging research faces t he challenge of detecting brain tumor through mri magnetic resonance image there is a high diversit y in the appearance of tumor tissue among different patients and in many cases similarity w ith the usual tissue we have used mri because it provide accurate visualize of anatomical structure of tissue in this paper various method that have been used for segmentation of mri for detecting brain tumor is reviewed keywords mri segmentation clustering k means algorithm fuzzy c mean algorithm edge detecti on thresholding region growing region splitting watershed segmentation algorithm entropy svm ', 
             'over the past years there has been an unpar alleled increase in understanding of cancer biology this trans formation is exempli ed by hanahan and weinberg s decisionin to expand their original hallmarks of cancer fromsix traits to ten at the same time mathematical modelinghas emerged as a natural tool for unraveling the complexprocesses that contribute to the initiation and progressionof cancer for testing hypotheses about experimental andclinical observations and assisting with the development ofnew approaches for improving its treatment this article startsby reviewing some of the earliest models of tumor growthand tumor responses to radiotherapy following hanahan andweinberg s lead attention then focuses on how closer col laboration with cancer scientists and access to experimentaldata are stimulating the development of new and increasinglydetailed models that account for example for tumor immuneinteractions and immunotherapy the article concludes by dis cussing the ways in which mathematical modeling is beingintegrated with experimental and clinical studies and outlininghow this could improve disease diagnosis and the delivery ofeffective personalized treatments to cancer patients as such this article serves as an', 
             'the internet of things iot is rapidly spreading reaching a multitude of different domains including personalhealth care environmental monitoring home automation smartmobility and industry as a consequence more and moreiot devices are being deployed in a variety of public and pri vate environments progressively becoming common objects ofeveryday life it is hence apparent that in such a scenario cybersecurity becomes critical to avoid threats like leakage ofsensible information denial of service dos attacks unautho rized network access and so on unfortunately many low endiot commercial products do not usually support strong secu rity mechanisms and can hence be target of or even meansfor a number of security attacks the aim of this article is toprovide a broad overview of the security risks in the iot sec tor and to discuss some possible counteractions to this end after a general', 'internet of things iot results in a massive amountof streaming data often referred to as big data which bringsnew opportunities to monitor agricultural and food processes besides sensors big data from social media is also becomingimportant for the food industry in this review we present anoverview of iot big data and arti cial intelligence ai andtheir disruptive role in shaping the future of agri food systems following an', 'mri imaging play an important role in brain tumor for analysis diagnosis and treatment planning it s helpful to doctor for determine the previous steps of brain tumor brain tumor detections are using mri images is a challenging task because the complex structure of the brain brain tumor is an abnormal growth of cell of brain mri images offer better difference concern of various soft tissues of human body mri image provides better results than ct ultrasound and x ray in this the various preprocessing post processing and methods like filtering contrast enhancement edge detection and post processing techniques like histogram threshold segmentation morphological operation through image processing ip tool is available in matlab for detection of brain tumor images mri images are discussed keywords brain tumor bt mri images ct ip x ray ', 'internet of things iot which will create a hugenetwork of billions or trillions of things communicatingwith one another are facing many technical and applicationchallenges this paper introduces the status of iot developmentin china including policies r d plans applications and stan dardization with china s perspective this paper depicts suchchallenges on technologies applications and standardization andalso proposes an open and general iot architecture consisting ofthree platforms to meet the architecture challenge finally thispaper discusses the opportunity and prospect of iot index t erms internet of things iot iot application iotarchitecture iot challenge iot standardization i i', 'digital twin network dtn is an emerging networkthat utilizes digital twin dt technology to create the virtualtwins of physical objects dtn realizes co evolution betweenphysical and virtual spaces through dt modeling communica tion computing data processing technologies in this article wepresent a comprehensive survey of dtn to explore the poten tiality of dt first we elaborate key features and de nitions ofdtn next the key technologies and the technical challenges indtn are discussed furthermore we depict the typical applica tion scenarios such as manufacturing aviation healthcare gnetworks intelligent transportation systems and urban intelli gence in smart cities finally the new trends and open researchissues related to dtn are pointed out index terms digital twin dt digital twin network dtn dt modeling i i', 'see discussions st ats and author pr ofiles f or this public ation at https www researchgate ne t public ation brain tu mou r detection using mri\nconf erence paper februar y citations\nreads author muhammad u sman\nuniv ersity of engineering and t echnolog y taxila publica tion citations see profile\nall c ontent f ollo wing this p age was uplo aded b y muhammad u sman on oct ober the user has r equest ed enhanc ement of the do wnlo aded file ', 'the successful early diagnosis of brain tumors plays a major role in improving the treatment outcomes and thusimproving patient survival manually evaluating the numerous magnetic resonance imaging mri imagesproduced routinely in the clinic is a di cult process thus there is a crucial need for computer aided methods with better accuracy for early tumor diagnosis computer aided brain tumor diagnosis from mri images consistsof tumor detection segmentation and classi cation processes over the past few years many studies have focused on traditional or classical machine learning techniques for brain tumor diagnosis recently interest hasdeveloped in using deep learning techniques for diagnosing brain tumors with better accuracy and robustness this study presents a comprehensive review of traditional machine learning techniques and evolving deeplearning techniques for brain tumor diagnosis this review paper identi es the key achievements re ected in the performance measurement metrics of the applied algorithms in the three diagnosis processes in addition thisstudy discusses the key ndings and draws attention to the lessons learned as a roadmap for future research ', 'decentralized paradigm in the eld of cybersecu rity and machine learning ml for the emerging internet ofthings iot has gained a lot of attention from the government academia and industries in recent years federated cyberse curity fc is regarded as a revolutionary concept to makethe iot safer and more ef cient in the future this emergingconcept has the potential of detecting security threats takingcountermeasures and limiting the spreading of threats over theiot network system ef ciently an objective of cybersecurity isachieved by forming the federation of the learned and sharedmodel on top of various participants federated learning fl which is regarded as a privacy aware ml model is particularlyuseful to secure the vulnerable iot environment in this arti cle we start with background and comparison of centralizedlearning distributed on site learning and fl which is then fol lowed by a survey of the application of fl to cybersecurity foriot this survey primarily focuses on the security aspect but italso discusses several approaches that address the performanceissues e g accuracy latency resource constraint and others associated with fl which may impact the security and over all performance of the iot to anticipate the future evolutionof this new paradigm we discuss the main ongoing researchefforts challenges and research trends in this area with this arti cle readers can have a more thorough understanding of fl forcybersecurity as well as cybersecurity for fl different securityattacks and countermeasures index terms cybersecurity data of oading federatedcybersecurity fc federated learning fl machinelearning ml i i', 'the latest g mobile networks have enabled manyexciting internet of things iot applications that employunmanned aerial vehicles ua vs drones the success of mostua v based iot applications is heavily dependent on arti cialintelligence ai technologies for instance computer vision andpath planning these ai methods must process data and providedecisions while ensuring low latency and low energy consumption however the existing cloud based ai paradigm nds it dif cultto meet these strict ua v requirements edge ai which runsai on device or on edge servers close to users can be suitablefor improving ua v based iot services this article provides acomprehensive analysis of the impact of edge ai on key ua vtechnical aspects i e autonomous navigation formation con trol power management security and privacy computer vision and communication and applications i e delivery systems civilinfrastructure inspection precision agriculture search and res cue sar operations acting as aerial wireless base stations bss and drone light shows as guidance for researchersand practitioners this article also explores ua v based edge aiimplementation challenges lessons learned and future researchdirections index terms arti cial intelligence ai edge ai edge com puting edge intelligence internet of things iot mec unmanned aerial vehicle ua v i i', 'see discussions st ats and author pr ofiles f or this public ation at https www researchgate ne t public ation a framework for ef cient brain tu mor classi cation using mri images\narticle in mathematic al bioscienc es and engineering mbe june doi mbe citations\nreads author s including some o f the author s of this public ation ar e also w orking on these r elat ed pr ojects ener gy aware dynamic workflo w applic ation p artitioning and t ask scheduling in he terogeneous mobile cloud ne twork view pr oject\nhigh v oltage out door insulat ors view pr oject\nmuhammad aamir\nhuangg ang normal univ ersity publica tions citations see profile\nziaur r ahman\nhuangg ang normal univ ersity publica tions citations see profile\nwaheed ahmed abr o\nsouthe ast univ ersity china publica tions citations see profile\nzaheer ahmed day o\nhuangg ang normal univ ersity publica tions citations see profile\nall c ontent f ollo wing this p age was uplo aded b y muhammad sho aib bhutt a on a ugust the user has r equest ed enhanc ement of the do wnlo aded file ', 'in this paper the automated classification of brain tumor grades using ffnn mlp and bpn are performed the features of the brain tumor grades are extracted using glcm and glrm the optimal features are selected using fuzzy entropy measure based on the features that are extracted from the various grades of brain tumor mris the classifiers are trained and tested the performances of the classifiers are evaluated in both the testing and training phases with various parameters these classifiers are tested using a dataset of mr brain images from the accuracy in classifying the tumor grades it is found that bpn outperforms other classifiers with the classification accuracy of and this will be the fruitful automated tool for the classification of brain tumors according to their grades keywords classifier feature extraction feature selection neural network statistical features', 'the augmented reality ar applications have beenwidely used in the eld of internet of things iot because ofgood immersion experience for users but their ultralow delaydemand and high energy consumption bring a huge challengeto the current communication system and terminal power theemergence of mobile edge computing mec provides a goodthinking to solve this challenge in this article we study anenergy ef cient task of oading and resource allocation schemefor ar in both the single mec and multi mec systems first a more speci c and detailed ar application model is establishedas a directed acyclic graph according to its internal function ality second based on this ar model a joint optimizationproblem of task of oading and resource allocation is formulatedto minimize the energy consumption of each user subject to thelatency requirement and the limited resources the problem isa mixed multiuser competition and cooperation problem whichinvolves the task of oading decision uplink downlink transmis sion resources allocation and computing resources allocation ofusers and mec server since it is an np hard problem and thecommunication environment is dynamic it is dif cult for geneticalgorithms or heuristic algorithms to solve therefore we proposean intelligent and ef cient resource allocation and task of oadingalgorithm based on the deep reinforcement learning frameworkof multiagent deep deterministic policy gradient maddpg in adynamic communication environment finally simulation resultsshow that the proposed algorithm can greatly reduce the energyconsumption of each user terminal index terms augmented reality ar deep reinforce ment learning internet of things iot mobile edge com puting mec multiagent deep deterministic policy gradient maddpg resource allocation task of oading i i', 'the proliferation of internet of things iot andthe success of rich cloud services have pushed the horizon ofa new computing paradigm edge computing which calls forprocessing the data at the edge of the network edge comput ing has the potential to address the concerns of response timerequirement battery life constraint bandwidth cost saving aswell as data safety and privacy in this paper we introduce thede nition of edge computing followed by several case studies ranging from cloud of oading to smart home and city as wellas collaborative edge to materialize the concept of edge com puting finally we present several challenges and opportunitiesin the eld of edge computing and hope this paper will gainattention from the community and inspire more research in thisdirection index terms edge computing internet of things iot smarthome and city i i', 'deeper neural networks are more dif cult to train we\npresent a residual learning framework to ease the training\nof networks that are substantially deeper than those used\npreviously we explicitly reformulate the layers as learn ing residual functions with reference to the layer inputs in stead of learning unreferenced functions we provide com prehensive empirical evidence showing that these residual\nnetworks are easier to optimize and can gain accuracy from\nconsiderably increased depth on the imagenet dataset we\nevaluate residual nets with a depth of up to layers \x02\ndeeper than vgg nets but still having lower complex ity an ensemble of these residual nets achieves error\non the imagenet testset this result won the st place on the\nilsvrc classi cation task we also present analysis\non cifar with and layers the depth of representations is of central importance\nfor many visual recognition tasks solely due to our ex tremely deep representations we obtain a relative im provement on the coco object detection dataset deep\nresidual nets are foundations of our submissions to ilsvrc coco competitions where we also won the st\nplaces on the tasks of imagenet detection imagenet local ization coco detection and coco segmentation ', 'the segmentation of brain tumors in magnetic resonance images mri is a challenging and difficult task because of the variety of their possible shapes locations i mage intensities in this paper it is intended to summarize and compare the methods of automatic detection of brain tumor through magnetic resonance image mri used in different stages of computer aided detection system cad brain image classification techniques are studied existing methods are classically divided into region based and contour based methods these are usually dedicated to full enhanced tumors or specific types of tumors the amount of resources required to describe large set of data is simplified and selected in for tissue segmentation in this paper modified image segmentation techniques were applied on mri scan images in order to detect brain tumors also in this paper a modified probabilistic neural network pnn mode l that is based on learning vector quantization lvq with image and data analysis and manipulation techniques is proposed to carry out an automated brain tumor classification using mri scans the assessment of the modified pnn classifier performance is me asured in terms of the training performance classification accuracies and computational time the simulation results showed that the modified pnn gives rapid and accurate classification compared with the image processing and published conventional pnn techniques simulation results also showed that the proposed system out performs the corresponding pnn system presented and successfully handle the process of brain tumor classification in mri image with accurac y index terms magnetic resonance image mri comput er aided detection system cad probabilistic neural network pnn edge detection', 'we present the design and evaluation of an integrated problem solving environment for cancer therapy analysis the\nenvironment intertwines a statistical martingale model and a k nearest neighbor approach with visual encodings including novel\ninteractive nomograms in order to compute and explain a patient s probability of survival as a function of similar patient results a\ncoordinated views paradigm enables exploration of the multivariate heterogeneous and few valued data from a large head and neck\ncancer repository a visual scaffolding approach further enables users to build from familiar representations to unfamiliar ones evaluation with domain experts show how this visualization approach and set of streamlined work ows enable the systematic and\nprecise analysis of a patient prognosis in the context of cohorts of similar patients we describe the design lessons learned from this\nsuccessful multi site remote collaboration index terms visual analytics precision medicine design studies nomograms parallel coordinate plots activity centered design ', 'the internet of things iot shall be able to incorpo rate transparently and seamlessly a large number of different andheterogeneous end systems while providing open access to selectedsubsets of data for the development of a plethora of digital services building a general architecture for the iot is hence a very complextask mainly because of the extremely large variety of devices linklayer technologies and services that may be involved in such asystem in this paper we focus speci cally to an urban iot systemthat while still being quite a broad category are characterized bytheir speci c application domain urban iots in fact are designedto support the smart city vision which aims at exploiting the mostadvanced communication technologies to support added valueservices for the administration of the city and for the citizens thispaper hence provides a comprehensive survey of the enablingtechnologies protocols and architecture for an urban iot fur thermore the paper will present and discuss the technical solutionsand best practice guidelines adopted in the padova smart cityproject a proof of concept deployment of an iot island in thecity of padova italy performed in collaboration with the citymunicipality index terms constrained application protocol coap ef cient xml interchange exi network architecture sensorsystem integration service functions and management smartcities testbed and trials lowpan ', 'the advantages of leveraging machine learning with quantum\ncomputing theory are still under study and very promising the new eld of quantum machine learning has great transformative potential in\nour eld however one of the major di\x0eculties presented for those in\neither the machine learning community and the quantum community is\nthe knowledge gap that naturally poses a barrier preventing both elds\nfrom joining forces without a signi\x0ccant amount of preparation time and space for professional development with this in mind we prepared\nthis succinct literature review that provides an', 'background with the publication of large studiesfrom different health systems comparing survival probabilities cancer registries are increasingly involved in clinical evaluationresearch the changing role of registries strictly depends on theintegration between the oncology system and proper informationtechnology it tools it is fundamental to improving validityand timeliness of data diffusion when both the number of sourceslinked and the number of variables registered are on the rise aims in this paper we present a modern web based managementsystem that allows to integrate different sources validate and elab orate data thus providing a new evaluation system for the oncologynetwork based on cancer registries materials and methods w e developed a web management system for the umbria can cer registry s g rtup based on ampax technology apache mysql php ajax and xml and object oriented programming iso iec standard is followed to ensure security accessto the information the s g rtup architecture is modular andextensible and information consistency is guaranteed by entity relationship principles cancer sites topology morphology andbehavior are coded according to the international classi cationof diseases classical epidemiological indices for a cancer registryare implemented incidence mortality years of potential life lost and cumulative risk s g rtup has tools to prepare data fortrend analysis and relative survival analysis geographical anal ysis is also implemented results s g rtup is integrated with theoncology network and gives timely epidemiological indices forevaluation of oncological activities the registration system thatwe developed can effectively manage different data sources au tomatic importing of routinely available data from pathologyarchives screening services and hospital discharge records willreduce the time needed to produce data and will also allow theexpansion of registered information several services for data vi sualization and statistical analysis are implemented a geographicinformation system based on google maps api is used for geolo calization of cases and map plotting of incidence and mortalityrates we implemented besag york and mollie s algorithm forreal time smoothed maps all services can be dynamically per formed over a subset of data that the user can select throughan innovative ltering system discussion and conclusion i tc o n tributed to shortening all phases of cancer registration includ ing linkage with external sources coding quality control data\nmanuscript received february revised june andseptember accepted december date of current versionoctober this work was supported by the department of health regional government of umbria this paper was recommended by editorm p fanti f bianconi v brunori and p valigi are with the department of elec tronic and information engineering university of perugia perugia italy e mail fortunato bianconi unipg it brunori valerio com valigi unipg it f la rosa and f stracci are with the department of surgical and medicalspecialties and public health university of perugia perugia italy e mail larosaf it fabs it color versions of one or more of the gures in this paper are available onlineat http ieeexplore ieee org digital object identi er tsmca management and analysis and publication of results integrationin the oncology network and secure web access allowed us todesign with clinicians innovative population based collaborativestudies our geographic analysis system enables us to developsophisticated dynamic geostatistic tools index terms cancer registry geographic information sys tems gis health care management information technologies oncology network evolution ', 'mri imaging play an important role in brain tumor for analysis diagnosis and treatment planning it s helpful to doctor for determine the previous steps of brain tumor brain tumor detections are using mri images is a challenging task because the complex structure of the brain brain tumor is an abnormal growth of cell of brain mri images offer better difference concern of various soft tissues of human body mri image provides better results than ct ultrasound and x ray in this the various preprocessing post processing and methods like filtering contrast enhancement edge detection and post processing techniques like histogram threshold segmentation morphological operation through image processing ip tool is available in matlab for detection of brain tumor images mri images are discussed keywords brain tumor bt mri images ct ip x ray ', 'in this paper first we will discuss the importance of physical insight in computational electromagnetics we explain why it is relatively easy to arrive at fast algorithms for static problems but harder for wave physics problems second the understanding of wave physics is very important in classical electromagnetics but its understanding also allows us to understand other physical phenomena requiring wave physics such as transport of electrons in complex structures last we discuss the casimir force calculation that needs a combine understanding of statistical physics quantum electromagnetics and classical electromagnetics computational electromagnetics can play an important role in casimir force calculation i i']

#####links
links = ['https://link.springer.com/article/10.1007/s11432-020-2881-9', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/Oncology-4.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/QuantumComputing-5.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/QuantumComputing-4.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/Oncology-5.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/Oncology-10.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/Oncology-7.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/QuantumComputing-6.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/QuantumComputing-7.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/Oncology-6.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/BrainTumor-9.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/Oncology-2.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/QuantumComputing-3.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/QuantumComputing-2.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/Oncology-3.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/BrainTumor-8.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/Oncology-1.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/IOT-8.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/IOT-9.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/BrainTumor-6.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/IOT-4.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/IOT-5.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/BrainTumor-7.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/BrainTumor-5.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/IOT-7.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/IOT-6.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/BrainTumor-4.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/BrainTumor-10.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/IOT-2.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/IOT-3.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/BrainTumor-1.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/BrainTumor-3.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/Oncology-8.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/IOT-1.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/QuantumComputing-8.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/Oncology-9.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/BrainTumor-2.pdf', '/Users/avinashgoli/Documents/BigDataProgramming/BigDataProject/DocumentDb/QuantumMechanics-1.pdf']
len(links)
linkdataDF = spark.createDataFrame([(links[i],removedStopWords[i])for i in range(len(links))],['link', 'docdata'])

             
keywords = 'oncosimulator'
removedStopWords = []
for i in documents:
    processedText = gensim.utils.simple_preprocess(i)
    t = remove_stopwords(' '.join(processedText))
    removedStopWords.append(t)
            

             
docs = spark.sparkContext.parallelize(removedStopWords).map(lambda line: line.split(" "))
# keywords = spark.sparkContext.parallelize(removedStopWords).map(lambda line: line.split(" "))

In [165]:
hashingTF = HashingTF(5000)
tf = hashingTF.transform(docs)
# print(tf.collect())

In [166]:
idf = IDF().fit(tf)
# idf.idf()

In [167]:
tfidf = idf.transform(tf)
print(type(tfidf))

<class 'pyspark.rdd.RDD'>


In [168]:
def cosine_dist(row):
    x = row[1][0]
    y = row[1][1]
    return (1 - x.dot(y)/(x.norm(2)*y.norm(2)))

In [175]:
keywordTF = hashingTF.transform(['cancer'])
print(keywordTF)
keywordHashValue = int(keywordTF.indices[0])
keywordHashValue


rdd1 = tfidf.zipWithIndex().map(lambda arg : (arg[1], arg[0]))
# print(rdd1.collect())
# rdd2 = keywordTF.zipWithIndex().map(lambda arg : (arg[1], arg[0]))
rdd2 = (keywordTF.indices[0], keywordTF.values[0])
# Join both RDDs.
rdd_joined = rdd1.join([rdd2])

res = rdd_joined.map(cosine_dist)
print(res.collect())

(5000,[315],[1.0])
(315, 1.0)


AttributeError: 'list' object has no attribute 'mapValues'

In [ ]:
keywordRelevance = tfidf.map(lambda x: x[keywordHashValue])
zippedResults = keywordRelevance.zip(docs)
print(keywordRelevance.collect())

In [ ]:
print('Printing results')
# print(zippedResults.collect())

In [ ]:
def to_dense(in_vec):
    return DenseVector(in_vec.toArray())


tokenizer = Tokenizer(inputCol="docdata", outputCol="words")
wordsData = tokenizer.transform(linkdataDF)
wordsData.head(5)


vectorizer = CountVectorizer(inputCol='words', outputCol='vectorizer').fit(wordsData)
wordsData = vectorizer.transform(wordsData)
wordsData.head(5)

idf = IDF(inputCol="vectorizer", outputCol="tfidf_features")
idf_model = idf.fit(wordsData)
wordsData = idf_model.transform(wordsData)
# wordsData.show()

to_dense_udf = udf(lambda x: to_dense(x), VectorUDT())

wordsData = wordsData.withColumn("tfidf_features_dense", to_dense_udf('tfidf_features'))
wordsData.select("link", "tfidf_features_dense", "words").show()